# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Thing?
# jupyter nbextension enable --py gmaps

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../Output/cities.csv')
cities_df.head()
cities_df.dtypes


City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure
gmaps.configure(api_key=g_key)

In [5]:
# Store 'Lat' and 'Lng' into  locations
locations_df = cities_df[['Lat','Lng']].astype(float)

# convert humidity to float
humidity_s = cities_df['Humidity'].astype(float)

In [6]:
# Create Humidity heatmap layer
fig_humidity = gmaps.figure()

head_layer = gmaps.heatmap_layer(locations_df, weights=humidity_s, dissipating=False, max_intensity=100, point_radius=1)

fig_humidity.add_layer(head_layer)
#fig_humidity

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# ideal conditions
# temp < 70
# cloudiness > 50
# humidity < 60

#ideal_df = cities_df[cities_df['Max Temp'] < 70 and cities_df['Cloudiness'] > 50 and cities_df['Humidity'] < 30]
ideal_df_1 = cities_df[cities_df['Max Temp'] < 23]
ideal_df_2 = ideal_df_1[ideal_df_1['Cloudiness'] > 25]
ideal_df = ideal_df_2[ideal_df_2['Humidity'] < 75]
ideal_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,57.000000,57.000000,57.000000,57.000000,57.000000,57.000000,5.700000e+01
mean,28.911053,27.192105,16.068596,60.368421,74.789474,3.355789,1.594539e+09
std,43.211472,90.619820,4.958194,12.355785,21.496459,2.028366,6.915573e+01
min,-54.800000,-149.900000,2.000000,9.000000,32.000000,0.460000,1.594539e+09
25%,-20.100000,-54.920000,14.000000,57.000000,58.000000,1.830000,1.594539e+09
50%,47.730000,30.790000,16.150000,63.000000,75.000000,2.900000,1.594539e+09
75%,65.200000,88.400000,20.000000,69.000000,98.000000,4.600000,1.594539e+09
max,78.220000,177.480000,22.910000,74.000000,100.000000,8.180000,1.594539e+09


In [8]:
# Max 10 pina
top10_df = ideal_df[:10]
top10_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,College,64.86,-147.80,17.00,63,40,1.50,US,1594538983
21,Ushuaia,-54.80,-68.30,2.00,74,39,7.70,AR,1594538988
27,Chokurdakh,70.63,147.92,22.91,43,100,4.34,RU,1594538989
33,Nikolskoye,59.70,30.79,18.89,67,75,3.00,RU,1594538991
34,Guerrero Negro,27.98,-114.06,22.59,60,93,5.44,MX,1594538991
43,Yar-Sale,66.83,70.83,16.61,60,86,2.22,RU,1594538993
45,Dalvik,65.97,-18.53,14.00,58,40,1.50,IS,1594538994
77,Constitución,-35.33,-72.42,8.99,71,100,1.83,CL,1594539002
79,Vostok,46.49,135.88,21.54,67,55,0.66,RU,1594539002
81,Neuquén,-38.95,-68.06,6.00,66,100,2.10,AR,1594538956


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = top10_df
hotel_df['Hotel Name'] = ''
hotel_df.head()

# Set params for hotel search
location = ''
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    'radius': '5000',
    'type': 'hotel',
    'key': g_key,
}

In [10]:
# using iterrows iterate through df
for index, row in hotel_df.iterrows():
    location = str(row['Lat']) + ', ' + str(row['Lng'])
    params['location'] = location
    response = requests.get(base_url,params=params).json()
    results = response['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except IndexError:
        pass

In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
1,College,64.86,-147.80,17.00,63,40,1.50,US,1594538983,Fairbanks
21,Ushuaia,-54.80,-68.30,2.00,74,39,7.70,AR,1594538988,Ushuaia
27,Chokurdakh,70.63,147.92,22.91,43,100,4.34,RU,1594538989,Chokurdakh
33,Nikolskoye,59.70,30.79,18.89,67,75,3.00,RU,1594538991,Saint Petersburg
34,Guerrero Negro,27.98,-114.06,22.59,60,93,5.44,MX,1594538991,Guerrero Negro
43,Yar-Sale,66.83,70.83,16.61,60,86,2.22,RU,1594538993,Yar-Sale
45,Dalvik,65.97,-18.53,14.00,58,40,1.50,IS,1594538994,Dalvik
77,Constitución,-35.33,-72.42,8.99,71,100,1.83,CL,1594539002,Constitución
79,Vostok,46.49,135.88,21.54,67,55,0.66,RU,1594539002,Work.Primorskiy GOK
81,Neuquén,-38.95,-68.06,6.00,66,100,2.10,AR,1594538956,Neuquen


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, display_info_box=True, info_box_content=hotel_info)

fig_humidity.add_layer(marker_layer)

# Display figure
fig_humidity


Figure(layout=FigureLayout(height='420px'))